In [1]:
import os
import cv2
import os.path as osp
import decord
import copy as cp
import numpy as np
# import matplotlib.pyplot as plt
import urllib
import moviepy.editor as mpy
# import random as rd

# from mmpose.apis import vis_pose_result
# from mmpose.models import TopDown
# from mmpose.models.pose_estimators import TopdownPoseEstimator as TopDown

from mmengine import load #, dump

from mmaction.datasets import (CenterCrop,
                               GeneratePoseTarget,
                               PoseCompact, PoseDecode,
                               Resize)

# We assume the annotation is already prepared
gym_ann_file = '../data/gym/gym_hrnet.pkl'  # https://download.openmmlab.com/mmaction/pyskl/data/gym/gym_hrnet.pkl
ntu60_ann_file = '../data/nturgbd/ntu60_hrnet.pkl'  # https://download.openmmlab.com/mmaction/pyskl/data/nturgbd/ntu60_hrnet.pkl

In [2]:
FONTFACE = cv2.FONT_HERSHEY_DUPLEX
FONTSCALE = 0.6
FONTCOLOR = (255, 255, 255)
BGBLUE = (0, 119, 182)
THICKNESS = 1
LINETYPE = 1

In [3]:
def add_label(frame, label, BGCOLOR=BGBLUE):
    threshold = 30
    def split_label(label):
        label = label.split()
        lines, cline = [], ''
        for word in label:
            if len(cline) + len(word) < threshold:
                cline = cline + ' ' + word
            else:
                lines.append(cline)
                cline = word
        if cline != '':
            lines += [cline]
        return lines
    
    if len(label) > 30:
        label = split_label(label)
    else:
        label = [label]
    label = ['Action: '] + label
    
    sizes = []
    for line in label:
        sizes.append(cv2.getTextSize(line, FONTFACE, FONTSCALE, THICKNESS)[0])
    box_width = max([x[0] for x in sizes]) + 10
    text_height = sizes[0][1]
    box_height = len(sizes) * (text_height + 6)
    
    cv2.rectangle(frame, (0, 0), (box_width, box_height), BGCOLOR, -1)
    for i, line in enumerate(label):
        location = (5, (text_height + 6) * i + text_height + 3)
        cv2.putText(frame, line, location, FONTFACE, FONTSCALE, FONTCOLOR, THICKNESS, LINETYPE)
    return frame
    

# def vis_skeleton(vid_path, anno, category_name=None, ratio=0.5):
#     vid = decord.VideoReader(vid_path)
#     frames = [x.asnumpy() for x in vid]
    
#     h, w, _ = frames[0].shape
#     new_shape = (int(w * ratio), int(h * ratio))
#     frames = [cv2.resize(f, new_shape) for f in frames]
    
#     assert len(frames) == anno['total_frames']
#     # The shape is N x T x K x 3
#     kps = np.concatenate([anno['keypoint'], anno['keypoint_score'][..., None]], axis=-1)
#     kps[..., :2] *= ratio
#     # Convert to T x N x K x 3
#     kps = kps.transpose([1, 0, 2, 3])
#     vis_frames = []

#     # we need an instance of TopDown model, so build a minimal one
#     model = TopDown(backbone=dict(type='ShuffleNetV1'))

#     for f, kp in zip(frames, kps):
#         bbox = np.zeros([0, 4], dtype=np.float32)
#         result = [dict(bbox=bbox, keypoints=k) for k in kp]
#         vis_frame = vis_pose_result(model, f, result)
        
#         if category_name is not None:
#             vis_frame = add_label(vis_frame, category_name)
        
#         vis_frames.append(vis_frame)
#     return vis_frames

In [4]:
keypoint_pipeline = [
    dict(type=PoseDecode),
    dict(type=PoseCompact, hw_ratio=1., allow_imgpad=True),
    dict(type=Resize, scale=(-1, 64)),
    dict(type=CenterCrop, crop_size=64),
    dict(type=GeneratePoseTarget, sigma=0.6, use_score=True, with_kp=True, with_limb=False)
]

limb_pipeline = [
    dict(type=PoseDecode),
    dict(type=PoseCompact, hw_ratio=1., allow_imgpad=True),
    dict(type=Resize, scale=(-1, 64)),
    dict(type=CenterCrop, crop_size=64),
    dict(type=GeneratePoseTarget, sigma=0.6, use_score=True, with_kp=False, with_limb=True)
]

from mmengine.dataset import Compose
def get_pseudo_heatmap(anno, flag='keypoint'):
    assert flag in ['keypoint', 'limb']
    pipeline = Compose(keypoint_pipeline if flag == 'keypoint' else limb_pipeline)
    return pipeline(anno)['imgs']

def vis_heatmaps(heatmaps, channel=-1, ratio=8):
    import matplotlib.cm as cm
    
    if ( 0 <= channel <= heatmaps.shape[1]-1 ):
        heatmaps = [heatmaps[x][channel] for x in range(heatmaps.shape[0])]
    else:  # draw all keypoints / limbs on the same map
        heatmaps = [np.max(x, axis=0) for x in heatmaps]
    
    h, w = heatmaps[0].shape    
    newh, neww = int(h * ratio), int(w * ratio)
    
    cmap = cm.viridis
    heatmaps = [(cmap(x)[..., :3] * 255).astype(np.uint8) for x in heatmaps]

    heatmaps = [cv2.resize(x, (neww, newh)) for x in heatmaps]
    return heatmaps

## GYM

In [ ]:
# Load GYM annotations
lines = list(urllib.request.urlopen('https://sdolivia.github.io/FineGym/resources/dataset/gym99_categories.txt'))
gym_categories = [x.decode().strip().split('; ')[-1] for x in lines]
gym_annos = load(gym_ann_file)['annotations']

In [ ]:
# download sample videos of GYM
!wget https://download.openmmlab.com/mmaction/posec3d/gym_samples.tar
!tar -xf gym_samples.tar
!rm gym_samples.tar

In [ ]:
gym_root = 'gym_samples/'
gym_vids = os.listdir(gym_root)
# visualize pose of which video? index in 0 - 50.
idx = 1
vid = gym_vids[idx]

frame_dir = vid.split('.')[0]
vid_path = osp.join(gym_root, vid)
anno = [x for x in gym_annos if x['frame_dir'] == frame_dir][0]

In [ ]:
vid_path

In [ ]:
# Visualize Skeleton
# vis_frames = vis_skeleton(vid_path, anno, gym_categories[anno['label']])
# vid = mpy.ImageSequenceClip(vis_frames, fps=24)
# vid.ipython_display()

In [ ]:
keypoint_heatmap = get_pseudo_heatmap(anno)
keypoint_mapvis = vis_heatmaps(keypoint_heatmap)
keypoint_mapvis = [add_label(f, gym_categories[anno['label']]) for f in keypoint_mapvis]
vid = mpy.ImageSequenceClip(keypoint_mapvis, fps=24)
vid.ipython_display()

In [ ]:
limb_heatmap = get_pseudo_heatmap(anno, 'limb')
limb_mapvis = vis_heatmaps(limb_heatmap)
limb_mapvis = [add_label(f, gym_categories[anno['label']]) for f in limb_mapvis]
vid = mpy.ImageSequenceClip(limb_mapvis, fps=24)
vid.ipython_display()

## NTU60

In [ ]:
# The name list of 
ntu_categories = ['drink water', 'eat meal/snack', 'brushing teeth', 'brushing hair', 'drop', 'pickup', 
                  'throw', 'sitting down', 'standing up (from sitting position)', 'clapping', 'reading', 
                  'writing', 'tear up paper', 'wear jacket', 'take off jacket', 'wear a shoe', 
                  'take off a shoe', 'wear on glasses', 'take off glasses', 'put on a hat/cap', 
                  'take off a hat/cap', 'cheer up', 'hand waving', 'kicking something', 
                  'reach into pocket', 'hopping (one foot jumping)', 'jump up', 
                  'make a phone call/answer phone', 'playing with phone/tablet', 'typing on a keyboard', 
                  'pointing to something with finger', 'taking a selfie', 'check time (from watch)', 
                  'rub two hands together', 'nod head/bow', 'shake head', 'wipe face', 'salute', 
                  'put the palms together', 'cross hands in front (say stop)', 'sneeze/cough', 
                  'staggering', 'falling', 'touch head (headache)', 'touch chest (stomachache/heart pain)', 
                  'touch back (backache)', 'touch neck (neckache)', 'nausea or vomiting condition', 
                  'use a fan (with hand or paper)/feeling warm', 'punching/slapping other person', 
                  'kicking other person', 'pushing other person', 'pat on back of other person', 
                  'point finger at the other person', 'hugging other person', 
                  'giving something to other person', "touch other person's pocket", 'handshaking', 
                  'walking towards each other', 'walking apart from each other']
ntu_annos = load(ntu60_ann_file)['annotations']

In [ ]:
# download sample videos of NTU-60
!wget https://download.openmmlab.com/mmaction/posec3d/ntu_samples.tar
!tar -xf ntu_samples.tar
!rm ntu_samples.tar

In [ ]:
ntu_root = 'ntu_samples/'
ntu_vids = os.listdir(ntu_root)
# visualize pose of which video? index in 0 - 50.
idx = 20
vid = ntu_vids[idx]

frame_dir = vid.split('.')[0]
vid_path = osp.join(ntu_root, vid)
anno = [x for x in ntu_annos if x['frame_dir'] == frame_dir.split('_')[0]][0]

In [ ]:
vid_path

In [ ]:
# vis_frames = vis_skeleton(vid_path, anno, ntu_categories[anno['label']])
# vid = mpy.ImageSequenceClip(vis_frames, fps=24)
# vid.ipython_display()

In [ ]:
keypoint_heatmap = get_pseudo_heatmap(anno)
keypoint_mapvis = vis_heatmaps(keypoint_heatmap)
keypoint_mapvis = [add_label(f, gym_categories[anno['label']]) for f in keypoint_mapvis]
vid = mpy.ImageSequenceClip(keypoint_mapvis, fps=24)
vid.ipython_display()

In [ ]:
limb_heatmap = get_pseudo_heatmap(anno, 'limb')
limb_mapvis = vis_heatmaps(limb_heatmap)
limb_mapvis = [add_label(f, gym_categories[anno['label']]) for f in limb_mapvis]
vid = mpy.ImageSequenceClip(limb_mapvis, fps=24)
vid.ipython_display()

# NTU60_2D_Single_Person

In [ ]:
ntu60_2d_ann_file = 'mmaction2/tools/data/skeleton/S001C001P001R001A001_rgb.pkl'

In [ ]:
ntu_categories = ['drink water', 'eat meal/snack', 'brushing teeth', 'brushing hair', 'drop', 'pickup', 
                  'throw', 'sitting down', 'standing up (from sitting position)', 'clapping', 'reading', 
                  'writing', 'tear up paper', 'wear jacket', 'take off jacket', 'wear a shoe', 
                  'take off a shoe', 'wear on glasses', 'take off glasses', 'put on a hat/cap', 
                  'take off a hat/cap', 'cheer up', 'hand waving', 'kicking something', 
                  'reach into pocket', 'hopping (one foot jumping)', 'jump up', 
                  'make a phone call/answer phone', 'playing with phone/tablet', 'typing on a keyboard', 
                  'pointing to something with finger', 'taking a selfie', 'check time (from watch)', 
                  'rub two hands together', 'nod head/bow', 'shake head', 'wipe face', 'salute', 
                  'put the palms together', 'cross hands in front (say stop)', 'sneeze/cough', 
                  'staggering', 'falling', 'touch head (headache)', 'touch chest (stomachache/heart pain)', 
                  'touch back (backache)', 'touch neck (neckache)', 'nausea or vomiting condition', 
                  'use a fan (with hand or paper)/feeling warm', 'punching/slapping other person', 
                  'kicking other person', 'pushing other person', 'pat on back of other person', 
                  'point finger at the other person', 'hugging other person', 
                  'giving something to other person', "touch other person's pocket", 'handshaking', 
                  'walking towards each other', 'walking apart from each other']
# ntu2d_annos = load(ntu60_2d_ann_file)['annotations']

In [ ]:
ntu2d_root = 'mmaction2/tools/data/skeleton/'
vid = 'S001C001P001R001A001_rgb.avi'

# frame_dir = vid.split('.')[0]
vid_path = osp.join(ntu2d_root, vid)
# anno = [x for x in ntu2d_annos if x['frame_dir'] == frame_dir.split('_')[0]][0]
anno = load(ntu60_2d_ann_file)

In [ ]:
keypoint_heatmap = get_pseudo_heatmap(anno)
keypoint_mapvis = vis_heatmaps(keypoint_heatmap, channel=-1)
keypoint_mapvis = [add_label(f, ntu_categories[anno['label']]) for f in keypoint_mapvis]
vid = mpy.ImageSequenceClip(keypoint_mapvis, fps=24)

# CHECK ndarray.shape, RESTART KERNEL if ERROR!
print(keypoint_heatmap.shape)  # frames X kpts X width X heighs

vid.write_videofile("S001C001P001R001A001_rgb_heatmap.mp4", remove_temp=True)
vid.ipython_display()

In [ ]:
limb_heatmap = get_pseudo_heatmap(anno, 'limb')
limb_mapvis = vis_heatmaps(limb_heatmap)
limb_mapvis = [add_label(f, ntu_categories[anno['label']]) for f in limb_mapvis]
vid = mpy.ImageSequenceClip(limb_mapvis, fps=24)

# CHECK ndarray.shape, RESTART KERNEL if ERROR!
print(limb_heatmap.shape[0])  # frames X kpts X width X heighs

vid.ipython_display()

# NTU60_2D_Multi_Persons

In [5]:
ntu60_2d_ann_file = 'mmaction2/tools/data/skeleton/DJI_0014_1080p_15r_10s.pkl'

In [6]:
ntu_categories = ['drink water', 'eat meal/snack', 'brushing teeth', 'brushing hair', 'drop', 'pickup', 
                  'throw', 'sitting down', 'standing up (from sitting position)', 'clapping', 'reading', 
                  'writing', 'tear up paper', 'wear jacket', 'take off jacket', 'wear a shoe', 
                  'take off a shoe', 'wear on glasses', 'take off glasses', 'put on a hat/cap', 
                  'take off a hat/cap', 'cheer up', 'hand waving', 'kicking something', 
                  'reach into pocket', 'hopping (one foot jumping)', 'jump up', 
                  'make a phone call/answer phone', 'playing with phone/tablet', 'typing on a keyboard', 
                  'pointing to something with finger', 'taking a selfie', 'check time (from watch)', 
                  'rub two hands together', 'nod head/bow', 'shake head', 'wipe face', 'salute', 
                  'put the palms together', 'cross hands in front (say stop)', 'sneeze/cough', 
                  'staggering', 'falling', 'touch head (headache)', 'touch chest (stomachache/heart pain)', 
                  'touch back (backache)', 'touch neck (neckache)', 'nausea or vomiting condition', 
                  'use a fan (with hand or paper)/feeling warm', 'punching/slapping other person', 
                  'kicking other person', 'pushing other person', 'pat on back of other person', 
                  'point finger at the other person', 'hugging other person', 
                  'giving something to other person', "touch other person's pocket", 'handshaking', 
                  'walking towards each other', 'walking apart from each other']
# ntu2d_annos = load(ntu60_2d_ann_file)['annotations']

In [7]:
ntu2d_root = '../'
vid = 'DJI_0014_1080p_15r_10s.MP4'

# frame_dir = vid.split('.')[0]
vid_path = osp.join(ntu2d_root, vid)
# anno = [x for x in ntu2d_annos if x['frame_dir'] == frame_dir.split('_')[0]][0]
anno = load(ntu60_2d_ann_file)

In [8]:
anno

{'keypoint': array([[[[1081.962   ,    7.154707],
          [ 978.6163  ,  129.04971 ],
          [ 965.3668  ,  123.74992 ],
          ...,
          [ 968.0167  ,  129.04971 ],
          [ 978.6163  ,   62.802425],
          [ 968.0167  ,  211.19635 ]],
 
         [[ 970.7613  ,  128.76625 ],
          [ 968.11725 ,  128.76625 ],
          [ 968.11725 ,  118.18997 ],
          ...,
          [ 965.4732  ,  128.76625 ],
          [ 976.04944 ,   73.24076 ],
          [ 968.11725 ,  208.0884  ]],
 
         [[ 968.8194  ,   89.015205],
          [ 976.6941  ,   94.26499 ],
          [ 958.3198  ,  120.51392 ],
          ...,
          [ 966.1945  ,  128.38861 ],
          [ 968.8194  ,  207.1354  ],
          [ 966.1945  ,  204.5105  ]],
 
         ...,
 
         [[ 567.41406 ,   10.817219],
          [ 581.8736  ,   25.276772],
          [ 563.79913 ,   10.817219],
          ...,
          [ 672.2458  ,  216.86586 ],
          [ 730.084   ,  318.08273 ],
          [ 730.084   ,  314.

In [9]:
keypoint_heatmap = get_pseudo_heatmap(anno)
keypoint_mapvis = vis_heatmaps(keypoint_heatmap, channel=-1)
# keypoint_mapvis = [add_label(f, ntu_categories[anno['label']]) for f in keypoint_mapvis]
vid = mpy.ImageSequenceClip(keypoint_mapvis, fps=15)

# CHECK ndarray.shape, RESTART KERNEL if ERROR!
print(keypoint_heatmap.shape)  # frames X kpts X width X heighs

# vid.write_videofile("S013C002P018R001A060_rgb_heatmap.mp4", remove_temp=True)
vid.ipython_display()

(150, 17, 64, 64)
Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4
